# Lab 2


`import interp` [scipy.interpolate.interp2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html)


## Objective

![theory](./Comparison_of_1D_and_2D_interpolation.svg)

## Code

In [2]:
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
from numba import jit

ModuleNotFoundError: No module named 'numba'

### Load the image

Load and show the image inline.

In [ ]:
image = Image.open('rice.tif')
image

### Convert the image to numpy array

In [ ]:
imarray = np.array(image)
print(imarray.shape)
print(imarray)

In [ ]:
print(imarray.shape[0])

### Nearest neighbor

In [ ]:
def Nearest_11812418(input_file, dim):
    image = Image.open(input_file)
    imarray = np.array(image)
    #find the dimenssion of input file
    input_height = imarray.shape[0]
    input_width = imarray.shape[1]

    #set the output
    #dim(0) is the height
    #dim(1) is the width
    #init an array
    output_arr = np.zeros(dim)

    #iterate in output array
    for i in range(dim[0]):
        for j in range(dim[1]):
            #i height (dim0)
            #j width (dim1)
            interpolation_h = round((i)*(input_height-1)/(dim[0]-1))
            interpolation_w = round((j)*(input_width-1)/(dim[1]-1))
            output_arr[i][j] = imarray[interpolation_h][interpolation_w]


    return output_arr

#### Testbench

In [ ]:
#enlarge
dim = (461,461)

start_time = time.time()
output_img = Nearest_11812418('rice.tif',dim)
print("--- %s seconds ---" % (time.time() - start_time))

im = Image.fromarray(output_img.astype(np.uint8))
im.save("Enlarged_Nearest_11812418.tif")
im.save("Enlarged_Nearest_11812418.png")
plt.imshow(output_img, cmap=plt.cm.Greys)
plt.show()


In [ ]:
#shrink
dim = (205,205)
output_img = Nearest_11812418('rice.tif',dim)
im = Image.fromarray(output_img.astype(np.uint8))
im.save("Shrinked_Nearest_11812418.tif")
im.save("Shrinked_Nearest_11812418.png")
plt.imshow(output_img, cmap=plt.cm.Greys)
plt.show()

### Bilinear Interpolation

In [ ]:
def Bilinear_11812418(input_file, dim):
    image = Image.open(input_file)
    imarray = np.array(image)
    #find the dimenssion of input file
    input_height = imarray.shape[0]
    input_width = imarray.shape[1]

    #set the output
    #dim(0) is the height
    #dim(1) is the width
    #init an array
    output_arr = np.zeros(dim)

    delta_height = 1
    delta_width = 1
    #base: output
    #iterate in output array
    for i in range(dim[0]):
        for j in range(dim[1]):
            #i height (dim0)
            #j width (dim1)

            #start from 1
            #transform to input coordinate
            projected_height = (i)*(input_height-1)/(dim[0]-1)
            projected_width = (j)*(input_width-1)/(dim[1]-1)
            #print(str(projected_height) + ' ' + str(projected_width))

            #find the border

            interpolation_h_up = int(np.ceil(projected_height))
            interpolation_h_down = int(np.floor(projected_height))
            interpolation_w_up = int(np.ceil(projected_width))
            interpolation_w_down = int(np.floor(projected_width))

            k_w_down = (abs(imarray[interpolation_h_up][interpolation_w_down]-imarray[interpolation_h_down][interpolation_w_down]))/delta_height

            w_down_val = imarray[interpolation_h_down][interpolation_w_down] + k_w_down*(projected_height-interpolation_h_down)

            k_w_up = (abs(imarray[interpolation_h_up][interpolation_w_up]-imarray[interpolation_h_down][interpolation_w_up]))/delta_height
            w_up_val = imarray[interpolation_h_down][interpolation_w_up] + k_w_up*(projected_height-interpolation_h_down)


            #special case
            if(projected_height % 1):
                w_down_val = imarray[int(projected_height)][int(projected_width)]
                w_up_val = imarray[int(projected_height)][int(projected_width)]

            if(projected_width % 1):
                interpolation_w_down = projected_width

            k_h = (abs(w_up_val-w_down_val)/delta_width)
            h_val = w_down_val + k_h*(projected_width-interpolation_w_down)

            output_arr[i][j] = h_val
    print(output_arr)
    return output_arr


#### Testbench

In [ ]:
dim = (461,461)

start_time = time.time()
output_img2 = Bilinear_11812418('rice.tif',dim)
print("--- %s seconds ---" % (time.time() - start_time))

im = Image.fromarray(output_img2.astype(np.uint8))
im.save("Enlarged_Bilinear_11812418.tif")
im.save("Enlarged_Bilinear_11812418.png")
plt.imshow(output_img2, cmap=plt.cm.Greys)
plt.show()

In [ ]:
dim = (205,205)
output_img2 = Bilinear_11812418('rice.tif',dim)
im = Image.fromarray(output_img2.astype(np.uint8))
im.save("Shrinked_Bilinear_11812418.tif")
im.save("Shrinked_Bilinear_11812418.png")
plt.imshow(output_img2, cmap=plt.cm.Greys)
plt.show()



In [ ]:
delta_img = output_img2-output_img

plt.imshow(delta_img, cmap=plt.cm.Greys)
plt.show()

### Bicubic with Scipy

[https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html)

In [ ]:
def Bicubic_11812418(input_file, dim):
    image = Image.open(input_file)
    imarray = np.array(image)

    #find the dimenssion of input file
    input_height = imarray.shape[0]
    input_width = imarray.shape[1]

    #set the output
    #dim(0) is the height
    #dim(1) is the width
    #init an array
    output_arr = np.zeros(dim)

    delta_height = 1
    delta_width = 1
    #base: output

    # construct the bicubic function
    def interp2d_bicubic_scipy(array_data,relative_y,relative_x):
        x = [0,1,2,3]
        y = [0,1,2,3]
        f = interpolate.interp2d(y, x, array_data, kind='cubic')
        interp_result = f(relative_y,relative_x)
        return interp_result

    #iterate in output array
    for i in range(dim[0]):
        for j in range(dim[1]):
            #i height (dim0)
            #j width (dim1)

            #start from 1
            #transform to input coordinate
            projected_height = (i)*(input_height-1)/(dim[0]-1)
            projected_width = (j)*(input_width-1)/(dim[1]-1)
            #print(str(projected_height) + ' ' + str(projected_width))

            #find the border
            interpolation_x_floor = int(np.floor(projected_width))
            interpolation_y_floor = int(np.floor(projected_height))

            array16 = []
            frame_position_x = interpolation_x_floor
            frame_position_y = interpolation_y_floor
            #special case
            #XY MIN
            if((interpolation_x_floor-1<0) or (interpolation_y_floor-1<0) or (interpolation_x_floor + 2>input_width-1) or (interpolation_y_floor + 2>input_height-1)):

                border_X_Min = max(0,interpolation_x_floor-1)
                border_Y_Min = max(0,interpolation_y_floor-1)
                border_X_Max = min(input_width-1,interpolation_x_floor+2)
                border_Y_Max = min(input_height-1,interpolation_y_floor+2)

                if ((interpolation_x_floor-border_X_Min)<(border_X_Max-interpolation_x_floor-1)):
                    #left border
                    frame_position_x = border_X_Min+1
                elif ((interpolation_x_floor-border_X_Min)>(border_X_Max-interpolation_x_floor-1)):
                    #right border
                    frame_position_x = border_X_Max-2
                if ((interpolation_y_floor-border_Y_Min)<(border_Y_Max-interpolation_y_floor-1)):
                    #lower border
                    frame_position_y = border_Y_Min+1
                elif ((interpolation_y_floor-border_Y_Min)>(border_Y_Max-interpolation_y_floor-1)):
                    #upper border
                    frame_position_y = border_Y_Max-2


            for k in range(4):
                array16 = np.concatenate((array16, imarray[frame_position_y -1 + k][frame_position_x -1 :frame_position_x + 3]), axis=None)
                


            output_arr[i][j] = interp2d_bicubic_scipy(array16,(projected_height-frame_position_y+1),(projected_width-frame_position_x+1))

    print(output_arr)
    return output_arr

#### Testbench

In [ ]:
dim = (461,461)
start_time = time.time()
output_img2 = Bicubic_11812418('rice.tif',dim)
print("--- %s seconds ---" % (time.time() - start_time))
im = Image.fromarray(output_img2.astype(np.uint8))

im.save("Enlarged_Bicubic_11812418.tif")
im.save("Enlarged_Bicubic_11812418.png")
plt.imshow(output_img2, cmap=plt.cm.Greys)
plt.show()

In [ ]:
dim = (205,205)
output_img2 = Bicubic_11812418('rice.tif',dim)
im = Image.fromarray(output_img2.astype(np.uint8))
im.save("Shrinked_Bicubic_11812418.tif")
im.save("Shrinked_Bicubic_11812418.png")
plt.imshow(output_img2, cmap=plt.cm.Greys)
plt.show()


In [ ]:
delta_img = output_img2-output_img

plt.imshow(delta_img, cmap=plt.cm.Greys)
plt.show()


### Biquintic with Scipy

[https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html)

In [ ]:
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import statistics


def Biquintic_11812418(input_file, dim):
    image = Image.open(input_file)
    imarray = np.array(image)

    #find the dimenssion of input file
    input_height = imarray.shape[0]
    input_width = imarray.shape[1]

    #set the output
    #dim(0) is the height
    #dim(1) is the width
    #init an array
    output_arr = np.zeros(dim)

    delta_height = 1
    delta_width = 1
    #base: output

    # construct the biquintic function
    def interp2d_biquintic_scipy(array_data,relative_y,relative_x):
        x = [0,1,2,3,4,5]
        y = [0,1,2,3,4,5]
        f = interpolate.interp2d(y, x, array_data, kind='quintic')
        interp_result = f(relative_y,relative_x)

        return interp_result

    #iterate in output array
    for i in range(dim[0]):
        for j in range(dim[1]):
            #i height (dim0)
            #j width (dim1)

            #start from 1
            #transform to input coordinate
            projected_height = (i)*(input_height-1)/(dim[0]-1)
            projected_width = (j)*(input_width-1)/(dim[1]-1)

            #find the border
            interpolation_x_floor = int(np.floor(projected_width))
            interpolation_y_floor = int(np.floor(projected_height))

            array36 = []
            frame_position_x = interpolation_x_floor
            frame_position_y = interpolation_y_floor
            #special case
            #XY MIN
            if((interpolation_x_floor-2<0) or (interpolation_y_floor-2<0) or (interpolation_x_floor + 2>input_width-2) or (interpolation_y_floor + 2>input_height-2)):

                border_X_Min = max(0,interpolation_x_floor-2)
                border_Y_Min = max(0,interpolation_y_floor-2)
                border_X_Max = min(input_width-1,interpolation_x_floor+3)
                border_Y_Max = min(input_height-1,interpolation_y_floor+3)

                if ((interpolation_x_floor-border_X_Min)<(border_X_Max-interpolation_x_floor-1)):
                    #left border
                    frame_position_x = border_X_Min+2
                elif ((interpolation_x_floor-border_X_Min)>(border_X_Max-interpolation_x_floor-1)):
                    #right border
                    frame_position_x = border_X_Max-3
                if ((interpolation_y_floor-border_Y_Min)<(border_Y_Max-interpolation_y_floor-1)):
                    #lower border
                    frame_position_y = border_Y_Min+2
                elif ((interpolation_y_floor-border_Y_Min)>(border_Y_Max-interpolation_y_floor-1)):
                    #upper border
                    frame_position_y = border_Y_Max-3


            for k in range(6):
                array36 = np.concatenate((array36, imarray[frame_position_y -2 + k][frame_position_x -2 :frame_position_x + 4]), axis=None)


            output_arr[i][j] = interp2d_biquintic_scipy(array36,(projected_height-frame_position_y+2),(projected_width-frame_position_x+2))

    print(output_arr)
    return output_arr

#### Testbench

In [ ]:
dim = (461,461)
start_time = time.time()
output_img2 = Biquintic_11812418('rice.tif',dim)
print("--- %s seconds ---" % (time.time() - start_time))
im = Image.fromarray(output_img2.astype(np.uint8))
im = Image.fromarray(output_img2.astype(np.uint8))
im.save("Enlarged_Biquintic_11812418.tif")
im.save("Enlarged_Biquintic_11812418.png")
plt.imshow(output_img2, cmap=plt.cm.Greys)
plt.show()

In [ ]:
dim = (205,205)
output_img2 = Biquintic_11812418('rice.tif',dim)
im = Image.fromarray(output_img2.astype(np.uint8))
im.save("Shrinked_Biquintic_11812418.tif")
im.save("Shrinked_Biquintic_11812418.png")
plt.imshow(output_img2, cmap=plt.cm.Greys)
plt.show()


In [ ]:
delta_img = output_img2-output_img

plt.imshow(delta_img, cmap=plt.cm.Greys)
plt.show()

